In [2]:
import easyaccess as ea
import pandas as pd
import numpy as np
import glob
import os

In [3]:
db = ea.connect(section='desoper')
cursor = db.cursor()

Connecting to DB ** desoper ** ...


In [9]:
newCandidate=pd.read_csv('/Volumes/lsa-gerdes/wsdiff_catalogs/season1/nofakes/cands_Y1/good_371.csv')

In [10]:
newCandidate

,date,ra,dec,expnum,exptime,band,ccd,mag,ml_score,objid,fakeid
0,2013/10/12 01:01:59,22:20:02.15,-0:44:20.5,243054,90.0,z,19,21.66,0.923,8355790,0
1,2013/10/12 00:59:59,22:20:02.15,-0:44:20.1,243053,90.0,i,19,21.57,0.930,8355689,0
2,2013/9/13 04:04:50,22:22:00.41,-0:19:18.3,233509,90.0,i,34,23.73,0.745,9977636,0
3,2013/9/8 03:20:34,22:22:25.16,-0:14:55.7,231500,90.0,g,55,21.32,0.930,4496229,0


In [13]:
#update the candidate list
candcmd="INSERT INTO scheibne.CANDLIST (CANID) \
VALUES ('S1_Y1_NF_Good_371')"

In [16]:
#update the linker
linkcmds=[]
for i in newCandidate.index:
    linkcmds+=["INSERT INTO scheibne.linker (CANID, OBJID) \
    VALUES ('S12_Y3_FO_Good_371', "+str(newCandidate['objid'][i])+")"]

In [17]:
#update the objects table
objcmds=[]
for i in newCandidate.index:
    dob="'"+newCandidate['date'][i]+"'"
    ra="'"+newCandidate['ra'][i]+"'"
    dec="'"+newCandidate['dec'][i]+"'"
    expnum=str(newCandidate['expnum'][i])
    exptime=str(newCandidate['exptime'][i])
    band="'"+newCandidate['band'][i]+"'"
    ccd=str(newCandidate['ccd'][i])
    mag=str(newCandidate['mag'][i])
    ml_score=str(newCandidate['ml_score'][i])
    objid=str(newCandidate['objid'][i])
    fakeid=str(newCandidate['fakeid'][i])
    objcmds+=["INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID) \
    VALUES ("+dob+","+ra+","+dec+","+expnum+","+exptime+","+band+","+ccd+","+mag+","+ml_score+","+objid+","+fakeid+")"]


In [37]:
for c in objcmds:
    print c

INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID)     VALUES ('2013/10/12 01:01:59','22:20:02.15','-0:44:20.5',243054,90.0,'z',19,21.66,0.923,8355790,0)
INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID)     VALUES ('2013/10/12 00:59:59','22:20:02.15','-0:44:20.1',243053,90.0,'i',19,21.57,0.93,8355689,0)
INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID)     VALUES ('2013/9/13 04:04:50','22:22:00.41','-0:19:18.3',233509,90.0,'i',34,23.73,0.745,9977636,0)
INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID)     VALUES ('2013/9/8 03:20:34','22:22:25.16','-0:14:55.7',231500,90.0,'g',55,21.32,0.93,4496229,0)


In [38]:
#updates the status

statcmd="INSERT INTO scheibne.status (CANID, STATUS, FLAG, MPC) \
 VALUES ('S1_Y1_NF_Good_371', 'Uninspected','No', 'Unreported')"

In [39]:
cursor.execute(candcmd)

In [40]:
for c in linkcmds:
    cursor.execute(c)

In [42]:
for c in objcmds:
    cursor.execute(c)

In [29]:
cursor.execute(statcmd)

In [43]:
def addCan(szn,cat,yr,nm):
    ncan=pd.read_csv('/Volumes/lsa-gerdes/wsdiff_catalogs/season'+str(szn)+'/'+cat+'/cands_Y'+str(yr)+'/'+nm+'.csv')
    if cat=='fakesonly':
        canid="S"+str(szn)+"Y"+str(yr)+"FO_"+nm
    else:
        canid="S"+str(szn)+"Y"+str(yr)+"NF_"+nm
    candcmd="INSERT INTO scheibne.CANDLIST (CANID) \
    VALUES ('"+canid+"')"
    linkcmds=[]
    for i in ncan.index:
        linkcmds+=["INSERT INTO scheibne.linker (CANID, OBJID) \
        VALUES ('"+canid+"', "+str(ncan['objid'][i])+")"]
    objcmds=[]
    for i in ncan.index:
        dob="'"+ncan['date'][i]+"'"
        ra="'"+ncan['ra'][i]+"'"
        dec="'"+ncan['dec'][i]+"'"
        expnum=str(ncan['expnum'][i])
        exptime=str(ncan['exptime'][i])
        band="'"+ncan['band'][i]+"'"
        ccd=str(ncan['ccd'][i])
        mag=str(ncan['mag'][i])
        ml_score=str(ncan['ml_score'][i])
        objid=str(ncan['objid'][i])
        if ncan['fakeid'][i]>=0:
            fakeid=str(ncan['fakeid'][i])
        else:
            fakeid=str(-99)
        objcmds+=["INSERT INTO scheibne.objects (DATE_OBS, RA, DEC, EXPNUM, EXPTIME, BAND, CCD, MAG , ML_SCORE, OBJID, FAKEID) \
        VALUES ("+dob+","+ra+","+dec+","+expnum+","+exptime+","+band+","+ccd+","+mag+","+ml_score+","+objid+","+fakeid+")"]
    statcmd="INSERT INTO scheibne.status (CANID, STATUS, FLAG, MPC) \
         VALUES ('"+canid+"', 'Uninspected','No', 'Unreported')"
    cursor.execute(candcmd)
    for l in linkcmds:
        try:
            cursor.execute(l)
        except:
            print l
    for o in objcmds:
        try:
            cursor.execute(o)
        except:
            print o
    cursor.execute(statcmd)

In [44]:
addCan(1,'nofakes',1,'good_371')

In [38]:
statcmd="INSERT INTO scheibne.status (CANID, STATUS, FLAG, MPC) \
 VALUES ('S12Y3_good_28', 'Uninspected','No', 'Unreported')"

In [39]:
cursor.execute(statcmd)

In [45]:
def bigAdd(szn,yr,cat):
    os.chdir('/Volumes/lsa-gerdes/wsdiff_catalogs/season'+str(szn)+'/'+cat+'/cands_Y'+str(yr))
        
    csv_list=glob.glob('*.csv')
    for c in csv_list:
        print c
        addCan(szn,cat,yr,c[:-4])

In [46]:
bigAdd(1,1,'nofakes')

good_295.csv
good_75.csv
good_229.csv
good_4.csv
good_91.csv
good_25.csv
good_236.csv
good_205.csv
good_283.csv
good_365.csv
good_23.csv
good_68.csv
good_105.csv
good_118.csv
good_321.csv
good_215.csv
good_158.csv
good_55.csv
good_333.csv
good_360.csv
good_221.csv
good_161.csv
good_350.csv
good_345.csv
good_149.csv
good_261.csv
good_314.csv
good_238.csv
good_266.csv
good_262.csv
good_254.csv
good_276.csv
good_3.csv
good_247.csv
good_217.csv
good_145.csv
good_138.csv
good_148.csv
good_371.csv
good_195.csv
good_361.csv
good_231.csv
good_297.csv
good_346.csv
good_36.csv
good_335.csv
good_51.csv
good_368.csv
good_114.csv
good_104.csv
good_18.csv
good_362.csv
good_131.csv
good_94.csv
good_120.csv
good_178.csv
good_42.csv
good_239.csv
good_93.csv
good_34.csv
good_64.csv
good_294.csv
good_312.csv
good_143.csv
good_130.csv
good_123.csv
good_358.csv
good_271.csv
good_234.csv
good_260.csv
good_339.csv
good_73.csv
good_109.csv
good_58.csv
good_227.csv
good_265.csv
good_204.csv
good_244.csv
good_3

In [65]:
ooplist=[4,13,45,20,37,6,39,22,31,14,33,10,11,8,30,34,42,18,36,3,23,25,27,29,28]

In [90]:
for p in range(0,50):
    cmd="delete from status where canid='S10Y2NF_good_"+str(p)+"'"
    cursor.execute(cmd)

True

In [8]:
namelist=[]
os.chdir('/Volumes/lsa-gerdes/wsdiff_catalogs/season1/nofakes/cands_Y1')
csv_list=glob.glob('*.csv')
for c in csv_list:
    namelist+=['S1Y1NF_'+c[:-4]]

In [9]:
print namelist

['S1Y1NF_good_295', 'S1Y1NF_good_75', 'S1Y1NF_good_229', 'S1Y1NF_good_4', 'S1Y1NF_good_91', 'S1Y1NF_good_25', 'S1Y1NF_good_236', 'S1Y1NF_good_205', 'S1Y1NF_good_283', 'S1Y1NF_good_365', 'S1Y1NF_good_23', 'S1Y1NF_good_68', 'S1Y1NF_good_105', 'S1Y1NF_good_118', 'S1Y1NF_good_321', 'S1Y1NF_good_215', 'S1Y1NF_good_158', 'S1Y1NF_good_55', 'S1Y1NF_good_333', 'S1Y1NF_good_360', 'S1Y1NF_good_221', 'S1Y1NF_good_161', 'S1Y1NF_good_350', 'S1Y1NF_good_345', 'S1Y1NF_good_149', 'S1Y1NF_good_261', 'S1Y1NF_good_314', 'S1Y1NF_good_238', 'S1Y1NF_good_266', 'S1Y1NF_good_262', 'S1Y1NF_good_254', 'S1Y1NF_good_276', 'S1Y1NF_good_3', 'S1Y1NF_good_247', 'S1Y1NF_good_217', 'S1Y1NF_good_145', 'S1Y1NF_good_138', 'S1Y1NF_good_148', 'S1Y1NF_good_371', 'S1Y1NF_good_195', 'S1Y1NF_good_361', 'S1Y1NF_good_231', 'S1Y1NF_good_297', 'S1Y1NF_good_346', 'S1Y1NF_good_36', 'S1Y1NF_good_335', 'S1Y1NF_good_51', 'S1Y1NF_good_368', 'S1Y1NF_good_114', 'S1Y1NF_good_104', 'S1Y1NF_good_18', 'S1Y1NF_good_362', 'S1Y1NF_good_131', 'S1Y1